In [ ]:
import pandas as pd

baseline_df = pd.read_csv("yolov8_counts_and_class.csv")
forecast_df = pd.read_csv("gnn_forecast_2025_2030.csv")

report = []

for image_id in forecast_df["image_id"].unique():
    baseline_row = baseline_df[baseline_df["image_id"] == image_id]
    if baseline_row.empty:
        continue
    
    baseline_veg = int(baseline_row["vegetation"].values[0])
    baseline_const = int(baseline_row["construction_zones"].values[0])

    for year in range(2025, 2031):
        row = forecast_df[(forecast_df["image_id"] == image_id) & (forecast_df["year"] == year)]
        if row.empty:
            continue

        current_veg = int(row["vegetation"].values[0]) if "vegetation" in row else max(baseline_veg - 1, 1) 
        current_const = int(row["construction_zones"].values[0])

        vlr = round((baseline_veg - current_veg) / baseline_veg, 2)
        ri = round((current_veg - baseline_veg) / baseline_veg, 2) if current_veg > baseline_veg else 0
        cgr = round(current_const / current_veg, 2) if current_veg != 0 else "inf"

        # Region-specific note (mocked for now)
        insight = "Recovery zone candidate" if ri > 0.15 else "High risk of degradation" if vlr > 0.25 else "Stable"

        report.append({
            "image_id": image_id,
            "year": year,
            "vegetation": current_veg,
            "construction_zones": current_const,
            "vegetation_loss_rate": vlr,
            "recovery_index": ri,
            "construction_to_green_ratio": cgr,
            "region_insight": insight
        })

pd.DataFrame(report).to_csv("results/environmental_impact_report.csv", index=False)
print("Environmental impact assessment saved to: results/environmental_impact_report.csv")
